In [98]:
# Environment and Core Libraries
import pandas as pd
import numpy as np
import random
import math
import matplotlib.pyplot as plt

# PennyLane and PyTorch
import pennylane as qml
import torch
from torch.nn import Module, ParameterDict, Parameter
from torch.optim import Adam

# Lambeq
import discopy
from lambeq import BobcatParser, Rewriter, IQPAnsatz, SpacyTokeniser, AtomicType
import spacy

# Patch for older discopy if needed (from your original code)
if not hasattr(getattr(discopy, "monoidal", None).Diagram, "is_mixed"):
    discopy.monoidal.Diagram.is_mixed = property(lambda self: False)

# Load spacy model
spacy.load('en_core_web_sm')

In [99]:
def load_data(csv_file, sample_fraction=1.0):
    sentences1, sentences2, is_duplicate = [], [], []
    try:
        df = pd.read_csv(csv_file, encoding='utf-8')
        if sample_fraction < 1.0:
            df = df.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)
        
        sentences1 = df['question1'].astype(str).tolist()
        sentences2 = df['question2'].astype(str).tolist()
        is_duplicate = df['is_duplicate'].tolist()
        
        print(f"Loaded {len(sentences1)} sentence pairs.")
        return sentences1, sentences2, is_duplicate
    except Exception as e:
        print(f"An error occurred: {e}")
        return [], [], []

In [ ]:
# ===============================================================
# 1. THE QNLP MODEL CLASS
# ===============================================================
class QNLPModel(Module):
    def __init__(self, symbols):
        super().__init__()
        self.params = ParameterDict({
            s.name.replace('.', '_'): Parameter(torch.rand(1) * 2 * torch.pi)
            for s in symbols
        })
    def forward(self, diagram):
        param_values = [self.params[s.name.replace('.', '_')] for s in diagram.free_symbols]
        if not param_values:
            return torch.tensor([])
        return torch.cat(param_values)

# ===============================================================
# 2. HELPER FUNCTIONS
# ===============================================================
def FischerInformation(Fidelity):
    """Calculates the Fubini-Study distance from the fidelity (overlap)."""
    # Use abs() to prevent math domain errors from floating point inaccuracies
    rootFidelity = math.sqrt(abs(Fidelity))
    # Clamp the value to the valid range [-1, 1] for acos to prevent errors
    clamped_val = max(-1.0, min(1.0, rootFidelity))
    return math.acos(clamped_val)

def get_diagram_width(diagram):
    if not diagram.boxes: return len(diagram.cod)
    return max([offset + len(box.dom) for box, offset in zip(diagram.boxes, diagram.offsets)] + [len(diagram.cod)])

def execute_discopy_diagram(diagram, params, wires):
    wire_map = {i: w for i, w in enumerate(wires)}
    param_idx = 0
    for gate, offset in zip(diagram.boxes, diagram.offsets):
        if hasattr(qml, gate.name):
            op = getattr(qml, gate.name)
            gate_params = []
            num_params = len(gate.free_symbols)
            if num_params > 0:
                gate_params = params[param_idx : param_idx + num_params]
                param_idx += num_params
            target_wires = [wire_map[i + offset] for i in range(len(gate.dom))]
            op(*gate_params, wires=target_wires)

# ===============================================================
# 3. PREPROCESSING FUNCTION
# ===============================================================
def preprocess_data_for_model(data_pairs, Tokeniser, ansatz, parser, rewriter, qubit_limit=20):
    # ... (This function remains the same as the previous version)
    print(f"Starting preprocessing with a qubit limit of {qubit_limit}...")
    filtered_pairs, all_symbols, n_max = [], set(), 0
    for s1, s2, is_duplicate in data_pairs:
        try:
            d1 = ansatz(rewriter(parser.sentence2diagram(Tokeniser.tokenise_sentence(s1), tokenised=True)))
            d2 = ansatz(rewriter(parser.sentence2diagram(Tokeniser.tokenise_sentence(s2), tokenised=True)))
            width1, width2 = get_diagram_width(d1), get_diagram_width(d2)
            if width1 <= qubit_limit and width2 <= qubit_limit:
                pair_data = {'s1': s1, 's2': s2, 'label': is_duplicate, 'd1': d1, 'd2': d2,
                             'structural_disparity': abs(len(d1.cod) - len(d2.cod))}
                filtered_pairs.append(pair_data)
                all_symbols.update(d1.free_symbols)
                all_symbols.update(d2.free_symbols)
                n_max = max(n_max, width1, width2)
        except Exception: continue
    print(f"Preprocessing complete. Found {len(filtered_pairs)} valid pairs.")
    print(f"Total unique parameters (symbols) found: {len(all_symbols)}")
    print(f"N_Max for the filtered dataset is: {n_max}")
    return filtered_pairs, sorted(list(all_symbols), key=lambda s: s.name), n_max

# ===============================================================
# 4. THE FINAL TRAINING FUNCTION
# ===============================================================
def train_model(data, symbols, n_max, base_learning_rate=0.01, lambda_penalty=0.1, epochs=10, use_dynamic_lr=False):
    model = QNLPModel(symbols)
    optimizer = Adam(model.parameters(), lr=base_learning_rate) 
    swap_dev = qml.device("lightning.qubit", wires=1 + 2 * n_max)
    
    @qml.qnode(swap_dev, interface="torch")
    def swap_test_qnode(params1, params2, diagram1, diagram2):
        qml.Hadamard(wires=0)
        execute_discopy_diagram(diagram1, params1, wires=range(1, 1 + n_max))
        execute_discopy_diagram(diagram2, params2, wires=range(1 + n_max, 1 + 2 * n_max))
        for i in range(n_max):
            qml.CSWAP(wires=[0, 1 + i, 1 + n_max + i])
        qml.Hadamard(wires=0)
        return qml.expval(qml.PauliZ(0))

    history = []
    
    if use_dynamic_lr:
        print("--- Starting training with CUSTOM DYNAMIC learning rate ---")
    else:
        print("--- Starting training with ADAM optimizer ---")

    for epoch in range(epochs):
        total_loss_epoch = 0
        num_trained_pairs = 0
        for i, pair in enumerate(data):
            optimizer.zero_grad()
            params1, params2 = model(pair['d1']), model(pair['d2'])
            if params1.nelement() == 0 and params2.nelement() == 0:
                continue
            
            num_trained_pairs += 1
            measured_overlap = swap_test_qnode(params1, params2, diagram1=pair['d1'], diagram2=pair['d2'])
            target_fidelity = pair['label']
            structural_disparity = pair['structural_disparity']
            
            fidelity_loss = (measured_overlap - target_fidelity)**2
            structural_penalty = lambda_penalty * structural_disparity
            total_loss = fidelity_loss + structural_penalty
            
            total_loss.backward()

            # --- OPTIMIZER CHOICE ---
            if use_dynamic_lr:
                # Option 1: Your custom dynamic learning rate
                qangle = FischerInformation(measured_overlap.item())
                normalized_qangle = qangle / (math.pi / 2)
                if target_fidelity == 1:
                    eta_dynamic = base_learning_rate * normalized_qangle
                else:
                    eta_dynamic = base_learning_rate * (1 - normalized_qangle)
                with torch.no_grad():
                    for p in model.parameters():
                        if p.grad is not None:
                            p.data.add_(-eta_dynamic * p.grad)
            else:
                # Option 2: Standard Adam Optimizer
                optimizer.step()
            
            total_loss_epoch += total_loss.item()
        
        avg_loss = total_loss_epoch / num_trained_pairs if num_trained_pairs > 0 else 0
        history.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Average Penalized Loss: {avg_loss:.4f}")
        
    return model, history

In [ ]:
def loss_function_manual(overlap, target_fidelity, structural_disparity, lambda_penalty):
    """
    Classical loss function used by the manual gradient calculator.
    """
    fidelity_loss = (target_fidelity - overlap)**2
    structural_penalty = lambda_penalty * structural_disparity
    return fidelity_loss + structural_penalty

def calculate_overlap_classical(qnode_func, params1, params2):
    """
    Calculates overlap on a statevector simulator (classical shortcut).
    """
    state1 = qnode_func(params1)
    state2 = qnode_func(params2)
    return np.abs(np.vdot(state1, state2))**2

def calculate_pair_gradients(qnode_func, params, s1, s2, target_fidelity, structural_disparity, lambda_penalty):
    """
    Calculates gradients for a pair of sentences using the parameter-shift rule.
    """
    gradients = {idx: 0.0 for idx in [(g, e) for g in range(len(params)) for e in range(len(params[g]))]}

    for group_idx, group in enumerate(params):
        for elem_idx in range(len(group)):
            param_index = (group_idx, elem_idx)

            # Shift the parameters
            params_plus = [p.copy() for p in params]
            params_plus[group_idx][elem_idx] += np.pi / 2
            
            params_minus = [p.copy() for p in params]
            params_minus[group_idx][elem_idx] -= np.pi / 2

            # Calculate overlap for shifted parameters
            # Note: This is a simplified gradient of a complex function. 
            # In a real scenario, the gradient of the loss with respect to
            # both state preparations would need to be calculated.
            overlap_plus = calculate_overlap_classical(qnode_func, params_plus, params_plus)
            overlap_minus = calculate_overlap_classical(qnode_func, params_minus, params_minus)

            loss_plus = loss_function_manual(overlap_plus, target_fidelity, structural_disparity, lambda_penalty)
            loss_minus = loss_function_manual(overlap_minus, target_fidelity, structural_disparity, lambda_penalty)
            
            # The parameter-shift rule
            grad = (loss_plus - loss_minus) / 2
            gradients[param_index] = grad

    return gradients

In [ ]:
def swap_test(state1_vec, state2_vec, num_qubits):
    """
    Performs a Quantum Swap Test between two quantum state vectors by sampling.
    """
    SHOT_COUNT = 100000  # High shot count for better statistical accuracy
    
    total_qubits = 1 + 2 * num_qubits  # 1 Ancilla qubit + 2 state registers
    dev = qml.device("default.qubit", wires=total_qubits, shots=SHOT_COUNT)

    @qml.qnode(dev)
    def circuit():
        # Step 1: Prepare the ancilla qubit in a superposition
        qml.Hadamard(wires=0)
        
        # Step 2: Prepare the two input states
        qml.StatePrep(state1_vec, wires=range(1, 1 + num_qubits))
        qml.StatePrep(state2_vec, wires=range(1 + num_qubits, 1 + 2 * num_qubits))

        # Step 3: Apply controlled-SWAP gates
        for i in range(num_qubits):
            qml.CSWAP(wires=[0, 1 + i, 1 + num_qubits + i])

        # Step 4: Apply Hadamard to the ancilla
        qml.Hadamard(wires=0)
        
        # Step 5: Measure the ancilla qubit
        return qml.sample(wires=0)

    # The squared overlap is related to the probability of measuring |0>
    # P(0) = (1 + |<psi|phi>|^2) / 2
    # Overlap^2 = 2 * P(0) - 1
    measurement_results = circuit()
    prob_0 = np.sum(measurement_results == 0) / SHOT_COUNT
    squared_overlap = 2 * prob_0 - 1
    
    return abs(squared_overlap)

In [ ]:
def plot_training_history(history):
    if not history:
        print("History is empty. Cannot plot.")
        return

    plt.figure(figsize=(10, 6))
    plt.plot(history, label='Average Loss per Epoch')
    plt.title('Training Loss Convergence')
    plt.xlabel('Epoch')
    plt.ylabel('Average Penalized Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# ===============================================================
#  HYPERPARAMETERS AND CONFIGURATION
# ===============================================================

# --- Data and Preprocessing ---
DATA_PATH = r'<ABSOLUTE_PATH_TO_YOUR_QUESTIONS.CSV>'
SAMPLE_FRACTION = 0.001  # Fraction of the dataset to use for a quick run
QUBIT_LIMIT = 12         # Max qubits for a diagram to be included (adjust based on RAM)

# --- Model and Circuit ---
# Define the mapping from grammatical types to qubits
N = AtomicType.NOUN
S = AtomicType.SENTENCE
OB_MAP = { N: 1, S: 1 }

# Define the ansatz
N_LAYERS = 1
ANSATZ = IQPAnsatz(OB_MAP, n_layers=N_LAYERS)

# Define the rewrite rules
REWRITE_RULES = ['curry', 'prepositional_phrase', 'determiner']

# --- Training ---
EPOCHS = 15
BASE_LEARNING_RATE = 0.01
LAMBDA_PENALTY = 0.1

In [ ]:
#Click this to run model
# 1. Initialize lambeq components using the defined hyperparameters
tokeniser = SpacyTokeniser()
parser = BobcatParser()
rewriter = Rewriter(REWRITE_RULES)

# 2. Load and preprocess data
sentences1, sentences2, value = load_data(DATA_PATH, sample_fraction=SAMPLE_FRACTION)
data_pairs = list(zip(sentences1, sentences2, value))

filtered_data, symbols, n_max = preprocess_data_for_model(
    data_pairs, tokeniser, ANSATZ, parser, rewriter, qubit_limit=QUBIT_LIMIT
)

# 3. Run training and plot results
if filtered_data and n_max > 0:
    trained_model, history = train_model(
        filtered_data, 
        symbols, 
        n_max, 
        epochs=EPOCHS, 
        base_learning_rate=BASE_LEARNING_RATE,
        lambda_penalty=LAMBDA_PENALTY
    )
    plot_training_history(history)
else:
    print("\nNo data to train on. Please check your QUBIT_LIMIT or SAMPLE_FRACTION.")